In [1]:
import random
import numpy as np

In [2]:
random.seed(42)

# SPDZ

## Parameters

In [3]:
#
# Use e.g. https://www.compilejava.net/
#
#import java.util.*;
#import java.math.*;
#
#public class Entrypoint
#{
#  public static void main(String[] args)
#  {
#    BigInteger q = BigInteger.probablePrime(128, new Random());    
#    System.out.println(q);
#  }
#}

BASE = 10
KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

# Q field
Q = 6497992661811505123 # < 64 bits
Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

# P field
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits
P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

## Fixed-point encoding

In [4]:
def encode(rational, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = int(rational * BASE**precision_fractional)
    field_element = upscaled % field
    return field_element

def decode(field_element, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = field_element if field_element <= field/2 else field_element - field
    rational = upscaled / BASE**precision_fractional
    return rational

In [5]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    y0 = x[0] // BASE**amount
    y1 = field - ((field - x[1]) // BASE**amount)
    return [y0, y1]

## Sharing and reconstruction

In [6]:
def share(secret, field=Q):
    first  = random.randrange(field)
    second = (secret - first) % field
    return [first, second]

def reconstruct(shares, field=Q):
    return sum(shares) % field

## Linear operations

In [7]:
def add(x, y, field=Q):
    return [ (xi + yi) % field for xi, yi in zip(x, y) ]

def sub(x, y, field=Q):
    return [ (xi - yi) % field for xi, yi in zip(x, y) ]

def add_public(x, k, field=Q):
    return add(x, [k, 0], field)

def sub_public(x, k, field=Q):
    return sub(x, [k, 0], field)

def mul_public(x, k, field=Q):
    return [ (xi * k) % field for xi in x ]

a = share(5)
b = share(7)
c = add(a, b)
assert reconstruct(c) == 12

## Multiplication

In [8]:
def generate_multiplication_triple(field=Q):
    a = random.randrange(field)
    b = random.randrange(field)
    c = a * b % field
    return share(a, field), share(b, field), share(c, field)

(a, b, c) = generate_multiplication_triple()
assert reconstruct(a) * reconstruct(b) % Q == reconstruct(c)

In [9]:
def mul(x, y, triple, field=Q):
    a, b, c = triple
    # local masking
    d = sub(x, a, field)
    e = sub(y, b, field)
    # communication: the players simultanously send their shares to the other
    delta = reconstruct(d, field)
    epsilon = reconstruct(e, field)
    # local combination
    r = delta * epsilon % field
    s = mul_public(a, epsilon, field)
    t = mul_public(b, delta, field)
    return add(s, add(t, add_public(c, r, field), field), field)

m = random.randrange(BOUND)
n = random.randrange(BOUND)
x = share(m)
y = share(n)
z = mul(x, y, generate_multiplication_triple())
assert reconstruct(z) == m*n

In [10]:
x = share(encode(-.5))

x2 = truncate(mul( x,  x, generate_multiplication_triple()), PRECISION_FRACTIONAL)
x4 = truncate(mul(x2, x2, generate_multiplication_triple()), PRECISION_FRACTIONAL)
y = decode(reconstruct(x4))
assert y == (-.5)**4

x2 = mul( x,  x, generate_multiplication_triple())
x4 = mul(x2, x2, generate_multiplication_triple())
y = decode(reconstruct(truncate(x4, 2*PRECISION_FRACTIONAL)))
assert not y == (-.5)**4

## Powering and polynomials

In [11]:
def generate_powering_triple(exponent, field=Q):
    a = random.randrange(field)
    return [ share(pow(a, e, field)) for e in range(1, exponent+1) ]

a, aa, aaa, aaaa = generate_powering_triple(4)
assert reconstruct(a) * reconstruct(a) % Q == reconstruct(aa) 
assert reconstruct(aa) * reconstruct(a) % Q == reconstruct(aaa)
assert reconstruct(aaa) * reconstruct(a) % Q == reconstruct(aaaa)

In [12]:
from functools import reduce
from scipy.misc import comb
binom = lambda n, k: comb(n, k, exact=True)

ONE = [1,0] # constant sharing of 1

def pows(x, triple, field=Q):
    # local masking
    a = triple[0]
    v = sub(x, a)
    # communication: the players simultanously send their share to the other
    epsilon = reconstruct(v)
    # local combination
    x_powers = []
    for exponent in range(1, len(triple)+1):
        # prepare all term values
        a_powers = [ONE] + triple[:exponent]
        e_powers = [ pow(epsilon, e, Q) for e in range(exponent+1) ]
        coeffs   = [ binom(exponent, k) for k in range(exponent+1) ]
        # compute and sum terms
        terms = ( mul_public(a, c * e) for a, c, e in zip(a_powers, coeffs, reversed(e_powers)) )
        x_powers.append(reduce(lambda x,y: add(x, y, field), terms))
    return x_powers

x = share(4)

xs = pows(x, generate_powering_triple(3))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,3+1) ]

xs = pows(x, generate_powering_triple(10))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,10+1) ]

In [14]:
def pol_public(x, coeffs, triple, field):
    powers = [ONE] + pows(x, triple, field)
    terms = ( mul_public(xe, ce, field) for xe,ce in zip(powers, coeffs) )
    return reduce(lambda y,z: add(y, z, field), terms)

x = share(5)
coeffs = [1,2,3,4]
y = pol_public(x, coeffs, generate_powering_triple(len(coeffs)-1, Q), Q)
assert reconstruct(y) == (1*pow(5,0) + 2*pow(5,1) + 3*pow(5,2) + 4*pow(5,3)) % Q

### Up and down sharing

In [15]:
def generate_statistical_mask():
    return random.randrange(2*BOUND * BASE**KAPPA)

def generate_zero_triple(field):
    return share(0, field)

def convert(x, from_field, to_field, zero_triple):
    # local mapping to positive representation
    x = add_public(x, BOUND, from_field)
    # local masking and conversion by player 0
    r = generate_statistical_mask()
    y0 = (zero_triple[0] - r) % to_field
    # exchange of masked share: one round of communication
    e = (x[0] + r) % from_field
    # local conversion by player 1
    xr = (e + x[1]) % from_field
    y1 = (zero_triple[1] + xr) % to_field
    # local combine
    y = [y0, y1]
    # local mapping back from positive representation
    y = sub_public(y, BOUND, to_field)
    return y

def upshare(x, large_zero_triple):
    return convert(x, Q, P, large_zero_triple)

def downshare(x, small_zero_triple):
    return convert(x, P, Q, small_zero_triple)

x = share(encode(-.5, Q), Q)
y = upshare(x, generate_zero_triple(P))
assert reconstruct(y, P) == encode(-.5, P)
z = downshare(y, generate_zero_triple(Q))
assert reconstruct(z, Q) == encode(-.5, Q)

In [16]:
x = share(encode(-.5, Q), Q)
x2 = mul(x, x, generate_multiplication_triple(Q), Q)
x4 = mul(x2, x2, generate_multiplication_triple(Q), Q)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, Q)
assert not decode(reconstruct(y, Q), Q) == (-.5)**4

x = share(encode(-.5, Q), Q)
x = upshare(x, generate_zero_triple(P))
x2 = mul(x, x, generate_multiplication_triple(P), P)
x4 = mul(x2, x2, generate_multiplication_triple(P), P)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, P)
y = downshare(y, generate_zero_triple(Q))
assert decode(reconstruct(y, Q), Q) == (-.5)**4

## Wrappers for working with NumPy

In [ ]:
class PrivateValue(object):
    
    def __init__(self, shares=None):
        self.shares = shares
    
    def unwrap(self):
        return self.shares
    
    def reconstruct(self):
        return PublicValue(reconstruct(self.shares))
    
    def __repr__(self):
        return "PrivateValue(%s)" % self.shares
    
    def __add__(x, y):
        if isinstance(y, PrivateValue):
            return PrivateValue(add(x.shares, y.shares))
        elif isinstance(y, PublicValue):
            return PrivateValue(add_public(x.shares, y.value))
        else:
            raise ValueError
    
    def __sub__(x, y):
        if isinstance(y, PrivateValue):
            return PrivateValue(sub(x.shares, y.shares))
        elif isinstance(y, PublicValue):
            return PrivateValue(sub_public(x.shares, y.value))
        else:
            raise ValueError
    
    def __mul__(x, y):
        if isinstance(y, PrivateValue):
            triple = generate_multiplication_triple()
            z = PrivateValue()
            z.shares = mul(x.shares, y.shares, triple)
            #z.shares = truncate(z.shares)
            assert reconstruct(z.shares) == reconstruct(x.shares) * reconstruct(y.shares) % Q
            return z
        elif isinstance(y, PublicValue):
            return PrivateValue(mul_public(x.shares, y.value))
        else:
            raise ValueError
    
class PublicValue(object):
    
    def __init__(self, value=None):
        self.value = value
    
    def unwrap(self):
        return self.value
    
    def share(self):
        return PrivateValue(share(self.value))
    
    def __repr__(self):
        return "PublicValue(%s)" % self.value
    
    def __add__(x, y):
        if isinstance(y, PublicValue):
            z = PublicValue()
            z.value = (x.value + y.value) % Q
            return z
        elif isinstance(y, PrivateValue):
            return y + x
        else:
            raise ValueError
            
    def __sub__(x, y):
        if isinstance(y, PublicValue):
            return PublicValue((x.value - y.value) % Q)
        else:
            raise ValueError
    
    def __mul__(x, y):
        if isinstance(y, PublicValue):
            z = PublicValue()
            z.value = (x.value * y.value) % Q
            return z
        elif isinstance(y, PrivateValue):
            return y * x
        else:
            raise ValueError
            
    def __eq__(x, y):
        if isinstance(y, PublicValue):
            return x.value == y.value
        else:
            raise ValueError
            
x = PrivateValue(share(5))
y = PrivateValue(share(2))
z = x * y
assert reconstruct(z.unwrap()) == 10

x = PrivateValue(share(5))
y = PublicValue(2)
z = x * y
assert reconstruct(z.unwrap()) == 10

In [ ]:
wrap = np.vectorize(lambda x: PublicValue(x))
unwrap = np.vectorize(lambda x: x.unwrap())

share_vec = np.vectorize(lambda x: x.share())
reconstruct_vec = np.vectorize(lambda x: x.reconstruct())

## Matrix multiplication

In [ ]:
def generate_matmul_triple(m, k, n):
    r = wrap(np.random.randint(Q, size=(m, k)))
    s = wrap(np.random.randint(Q, size=(k, n)))
    t = np.dot(r, s)
    return share_vec(r), share_vec(s), share_vec(t)

In [ ]:
def matmul(x, y, triple):
    r, s, t = triple
    rho = reconstruct_vec(x - r)
    sigma = reconstruct_vec(y - s)
    return np.dot(rho, sigma) + np.dot(r, sigma) + np.dot(rho, s) + t

m, k, n = 2, 5, 9
x = wrap(np.random.randint(Q, size=(m, k)))
y = wrap(np.random.randint(Q, size=(k, n)))

f = share_vec(x)
g = share_vec(y)
h = matmul(f, g, generate_matmul_triple(m, k, n))

zero = wrap(np.zeros(m*n, dtype=int).reshape(m,n))
assert (np.dot(x, y) - reconstruct_vec(h) == zero).all()

## Convolution

# Dump

## Security of mul

In [ ]:
a, b, c = generate_multiplication_triple(Q)

x = share(3, Q)
y = share(4, Q)

In [ ]:
e = sub(x, a, Q)
d = sub(y, b, Q)

epsilon = reconstruct(e, Q)
delta = reconstruct(d, Q)

r = epsilon * delta % Q
s = scalar_mul(b, epsilon, Q)
t = scalar_mul(a, delta, Q)

z = add(s, add(t, scalar_add(c, r, Q), Q), Q)

In [ ]:
z0_real = (c[0] + b[0] * epsilon + a[0] * delta + epsilon * delta) % Q
z0_sim = (12 - z[1]) % Q

assert z0_real == z0_sim

## Security of add

In [ ]:
x = share(2, Q)
y = share(5, Q)
z = add(x, y, Q)

In [ ]:
z0_real = (x[0] + y[0]) % Q
z0_sim = (7 - z[1]) % Q

assert z0_real == z0_sim

# Tensor

In [ ]:
class SecureScalar(object):

    def __init__(self):
        pass
    
    def __add__(x, y):
        pass
        

In [ ]:
class SecureTensor(object):
    
    def __init__(self, values=None):
        self.values = values
    
    def add(x, y):
        z = SecureTensor()
        z.values = x.values + y.values
        return z
    
    def __add__(x, y):
        return SecureTensor.add(x, y)
        
x = SecureTensor(np.arange(4).reshape(2,2))
y = SecureTensor(np.arange(4).reshape(2,2))
z = x.add(y)
z.values

In [ ]:
a = np.array([SecureScalar()])
b = np.arange(8).reshape(4,2)

print(isinstance(a.dtype, SecureScalar))

print(a.dot(b))
print(a[0,:])
print(a[:,0])
print(a.transpose(1,0)[0])

def add(x, y):
    print(x, y)
    return x + 2*y

vector_add = np.vectorize(add)
c = vector_add(a, b)